In [1]:
import requests
import json
import csv

url = "https://www.sjvair.com/api/1.0/monitors/"
county_list = ['Stockton' ,'Manteca' , 'Modesto' , 'Delano', 'Bakersfield', 'Empire', 'Waterford', 'Ceres' ,'Turlock' ,'Grayson' ,'Patterson' ,'Merced' ,'Fairmead' ,'Madera', 'Clovis', 'Hanford' ,'Corcoran' ,'Visalia' ,'Terra' ,'Bella']
loop_county_list = [i for i in county_list]
r = requests.get(url , params = {"is_sjvair": "true" , "county":json.dumps(loop_county_list), "device":"PurpleAir"})
with open("data.json" , "w") as i:
    i.write(json.dumps(r.json() , indent = 4))

with open("data.json", "r") as read_file:
    data = json.load(read_file)

sensor_list = ["Clovis RAMP PA" , "Fresno CARB CCAC" , "Fresno CARB CCAC" , "CCAC_Office" , "ucm-88d4" , "Madison Out" , "Fresno-Pacific RAMP PA" , "Fresno-Foundry RAMP PA"
        , "ucm-8b6f" , "Hanford RAMP PA" , "Corcoran-Patterson RAMP PA" , "The Birds" , "Ms.Roosevelt" , "ucm-1c8a" , "Terra Bella Elementary School" , "Ducor Union Elementary School",
        "Albany Park School" , "Terrace Elementary School" , "Morningside Elementary School" , "Almond Tree Middle School" , "Fremont Elementary School" , "CCAC BAM - Delano" , "DELANO_Coloc",
        "Harvest and La Vina School" , "Pioneer School" ," Nueva Vista Language Academy" , "Del Vista Math and Science Academy" ," Princeton Street School" , "Bakersfield Golden RAMP PA" , 
        "Bakersfield CARB CCAC"
        ]

new_list = []
print(len(data["data"]))
for index in range(0  ,len(data['data'])):
    if data['data'][index]['name'] in sensor_list:
        new_list.append({
            'name': data['data'][index]['name'],
            'id': data['data'][index]['purple_id'],
            'lat': data['data'][index]['position']['coordinates'][1],
            'log': data['data'][index]['position']['coordinates'][0],
        })
with open("new_data.json" , "w") as i:
    i.write(json.dumps(new_list , indent = 4))



In [4]:
import requests
import json
import csv

r = requests.get("https://www.sjvair.com/api/1.0/monitors/JAJWJnroQCSJz0Dr9uVC1g/entries/csv/")
with open('new_data.csv', 'w') as f:
    writer = csv.writer(f)
    for line in data.iter_lines():
        writer.writerow(line.decode('utf-8').split(','))


In [10]:
import requests
import json

r = requests.get("https://api.purpleair.com/v1/sensors" , headers = {'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"} , params= {
    "fields": ["latitude , longitude , name"],
    "nwlat": 37.471102,
    "nwlng": -122.173029,
    "selat": 37.293546,
    "selng": -122.108360
})
with open("palo.json" , "w") as i:
    i.write(json.dumps(r.json() , indent = 4))

In [11]:
import pandas as pd
import csv
import json

########### Read Json File ###############
with open("merced.json" , "r") as f:
    data_json = json.load(f)

########### Implementation the row name ########
fields = ['Sensor ID' , 'Name' , 'Latitude' , 'Longtitude']

########### Create paloalto.csv then write the list of data into csv ##########
with open('merced.csv' , 'w' , newline='') as f:
    w = csv.writer(f)
    w.writerow(fields)
    w.writerows(list(data_json['data']))

In [1]:
import pandas as pd
import csv
import requests
import json
import datetime
import time

date = datetime.date(2023, 8, 22)
date1 = datetime.date(2023 , 8 , 31)
start_date = time.mktime(date.timetuple())
end_date = time.mktime(date1.timetuple())

api_headers = {'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"}

data = pd.read_csv("fresno.csv")

data_frame = pd.DataFrame(data)

sensor_id = data_frame['Sensor ID']

i = 0

while i < len(sensor_id):
    sensor_id_int = int(sensor_id[i])
    r=requests.get(f"https://api.purpleair.com/v1/sensors/{sensor_id_int}/history/csv" , headers=api_headers , params= {
        "start_timestamp": start_date,
        "average": 60,
        "end_timestamp": end_date,
        "fields": ["pm2.5_alt"]
    })
    with open(f"fresno_folder/{sensor_id_int}.csv" , "wb") as f:
        f.write(r.content)
    i = i + 1


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import csv
import requests
import json
import datetime
import time

class GetPurpleAirData:
    #constructor
    csv_file = []
    folder_name = []
    headers = {}
    date = 0
    start_date = 0
    
    def __init__ (self , csv_file, folder_name , headers ,start_date , end_date):
        self.csv_file = csv_file
        self.folder_name = folder_name
        self.headers = headers
        self.start_date = start_date
        self.end_date = end_date
    def get_data(self):
        self.start_date = time.mktime(datetime.date(2023, 8, 22).timetuple())
        self.end_date = time.mktime(datetime.date(2023, 8, 31).timetuple())
        df = pd.DataFrame(pd.read_csv(self.csv_file))
        sensor_id = df['Sensor ID']
        for i in range(len(sensor_id)):
            sensor_id_int = int(sensor_id[i])
            r=requests.get(f"https://api.purpleair.com/v1/sensors/{sensor_id_int}/history/csv" , headers= self.headers , params= {
                "start_timestamp": self.start_date,
                "average": 60,
                "end_timestamp": self.end_date,
                "fields": ["pm2.5_alt"]
            })
            with open(f"{self.folder_name}/{sensor_id_int}.csv" , "wb") as f:
                f.write(r.content)

In [ ]:
start_date = time.mktime(date.timetuple())
end_date = time.mktime(date1.timetuple())

api_headers = {'X-API-Key': "5E56E968-4C34-11EE-A77F-42010A800009"}

data = pd.read_csv("sanjose.csv")

data_frame = pd.DataFrame(data)

sensor_id = data_frame['Sensor ID']

i = 0

while i < len(sensor_id):
    sensor_id_int = int(sensor_id[i])
    r=requests.get(f"https://api.purpleair.com/v1/sensors/{sensor_id_int}/history/csv" , headers=api_headers , params= {
        "start_timestamp": start_date,
        "average": 60,
        "end_timestamp": end_date,
        "fields": ["pm2.5_alt"]
    })
    with open(f"sanjose_folder/{sensor_id_int}.csv" , "wb") as f:
        f.write(r.content)
    i = i + 1